## Import Libraries

In [ ]:
import pandas as pd
import numpy as np 
from tqdm import tqdm
from PIL import Image
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers , models
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import VGG16 , ResNet50

In [52]:
folder_name = 'ExDark_Converted'

### Create train dataset

In [53]:
df = pd.DataFrame(data= None , columns= ['label' , 'filename'])
dataset_path = os.getcwd().replace('notebook' , '') + folder_name
count = 0
for folder in [i for i in os.listdir(dataset_path) if i not in ['.DS_Store' , 'label.csv']]:
    for i , file in enumerate(os.listdir(dataset_path + '/' + folder)):
            if file.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".tiff")):
                  df.loc[count + i] = [folder, file]
            count += len(os.listdir(dataset_path + '/' + folder))
df.to_csv(os.getcwd().replace('notebook' , '') + folder_name + '/' + 'label.csv')

## Pre Processing

In [ ]:
label_df_path = os.getcwd().replace('notebook' , '') + folder_name + '/' + 'label.csv' 
df = pd.read_csv(label_df_path , index_col=0)

# One Hot Encode genre file
df_encoded = pd.get_dummies(df['label'], columns=['label'])

y = df_encoded.values
x_train , x_test , y_train , y_test = train_test_split(x , y , test_size= 0.2)

## Modelling

### VGG16

#### Image Processing

In [ ]:
from tensorflow.keras.applications.vgg16 import preprocess_input

x = []
for i in tqdm(range(len(df))):
    label = df['label'].iloc[i]
    filename = df['filename'].iloc[i]
    filepath = os.path.join(os.getcwd().replace('notebook', ''), folder_name, label, filename)
    img = image.load_img(filepath, target_size=(224, 224))  # RGB
    
    # Convert to array and add batch dimension
    img_array = image.img_to_array(img) 
    img_array = np.expand_dims(img_array, axis=0)

    # Apply VGG-specific preprocessing (RGB → BGR + mean subtraction)
    img_preprocessed = preprocess_input(img_array)
    x.append(img)
x = np.array(x)

100%|██████████| 594/594 [00:00<00:00, 1305.88it/s]


### Tranfer Learning weights freezed

In [102]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(4096, activation='relu'),
    layers.Dense(4096, activation='relu'),
    layers.Dense(5, activation='softmax')
])
model.summary()

Model: "sequential_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_14 (Flatten)            │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_42 (Dense)                │ (None, 4096)           │   102,764,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_43 (Dense)                │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_44 (Dense)                │ (None, 5)              │        20,485 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 134,281,029 (512.24 MB)

 Trainable params: 119,566,341 (456.11 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [103]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss = 'categorical_crossentropy', metrics=['categorical_accuracy'])

In [104]:
est = model.fit(x_train,y_train , epochs=20 , batch_size=32 , validation_data=(x_test,y_test))

Epoch 1/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 37s 2s/step - categorical_accuracy: 0.2972 - loss: 362.6964 - val_categorical_accuracy: 0.4958 - val_loss: 1.6106
Epoch 2/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 35s 2s/step - categorical_accuracy: 0.6173 - loss: 22.9664 - val_categorical_accuracy: 0.6218 - val_loss: 1.2663
Epoch 3/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 36s 2s/step - categorical_accuracy: 0.7248 - loss: 1.6940 - val_categorical_accuracy: 0.5714 - val_loss: 1.2667
Epoch 4/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 36s 2s/step - categorical_accuracy: 0.8013 - loss: 0.7774 - val_categorical_accuracy: 0.6639 - val_loss: 1.3395
Epoch 5/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 36s 2s/step - categorical_accuracy: 0.8269 - loss: 0.6953 - val_categorical_accuracy: 0.6975 - val_loss: 1.4167
Epoch 6/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 36s 2s/step - categorical_accuracy: 0.8811 - loss: 0.6558 - val_categorical_accuracy: 0.6387 - val_loss: 4.9747
Epoch 7/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 36s 2s/step - categorical_accuracy: 0.9008 - loss: 0.7962 -

### Transfer Learning Wieghts Unfreeze

In [97]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = True
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(4096, activation='relu'),
    layers.Dense(4096, activation='relu'),
    layers.Dense(5, activation='softmax')
])
model.summary()

Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_13 (Flatten)            │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_39 (Dense)                │ (None, 4096)           │   102,764,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_40 (Dense)                │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_41 (Dense)                │ (None, 5)              │        20,485 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 134,281,029 (512.24 MB)

 Trainable params: 134,281,029 (512.24 MB)

 Non-trainable params: 0 (0.00 B)

In [100]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss = 'categorical_crossentropy', metrics=['categorical_accuracy'])

In [101]:
est = model.fit(x_train,y_train , epochs=20 , batch_size=32 , validation_data=(x_test,y_test))

Epoch 1/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 102s 7s/step - categorical_accuracy: 0.1987 - loss: 465.9507 - val_categorical_accuracy: 0.2185 - val_loss: 71.8047
Epoch 2/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 97s 6s/step - categorical_accuracy: 0.2076 - loss: 85.3738 - val_categorical_accuracy: 0.2605 - val_loss: 36.0992
Epoch 3/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 96s 6s/step - categorical_accuracy: 0.1643 - loss: 42.2553 - val_categorical_accuracy: 0.2185 - val_loss: 24.6875
Epoch 4/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 96s 6s/step - categorical_accuracy: 0.2273 - loss: 28.7070 - val_categorical_accuracy: 0.1261 - val_loss: 37.2346
Epoch 5/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 97s 7s/step - categorical_accuracy: 0.2066 - loss: 20.4467 - val_categorical_accuracy: 0.1261 - val_loss: 10.6890
Epoch 6/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 98s 7s/step - categorical_accuracy: 0.2036 - loss: 11.2328 - val_categorical_accuracy: 0.2605 - val_loss: 14.7428
Epoch 7/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 96s 6s/step - categorical_accuracy: 0.2070 - los